In [1]:
# importando as bibliotecas 
import os  #Interação com o sistema operacional
import numpy as np  #Operações numéricas e matrizes
import cv2  #Processamento de imagem
from tqdm import tqdm  #Barra de progresso
from tensorflow.keras.preprocessing.image import ImageDataGenerator  #Aumento de imagem
from sklearn.model_selection import train_test_split  # Divisão de dados em treino e teste
from tensorflow.keras.applications import VGG16  #Arquitetura de rede neural
from tensorflow.keras.models import Sequential, load_model  #Construção e carregamento de modelos
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D  #Camadas da rede neural
from tensorflow.keras.optimizers import Adam  #Otimizador
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  #Callbacks para treinamento
from sklearn.preprocessing import LabelEncoder  #Codificação de rótulos


In [2]:
# Definindo paths das pastas
# Caminhos das pastas com os dados. Especifica a variável base_path como o diretório principal.
base_path = 'C:/Users/andre/OneDrive/Desktop/Fiap/Kaggle/Chest_X-Ray'
train_dir = os.path.join(base_path, 'treino')
test_dir = os.path.join(base_path, 'teste')
val_dir = os.path.join(base_path, 'val')

In [3]:
# Parâmetros de preprocessamento
image_size = 224  # Tamanho das imagens
batch_size = 32

# image_size = 224: isso significa que todas as imagens serão redimensionadas para 224x224 pixels. 
# Independentemente do tamanho original da imagem, ela será adaptada para essa dimensão durante o pré-processamento.
# O processo é feito para garantir consistência no tamanho das imagens, o que facilita o 
# treinamento do modelo e garante que todas as entradas tenham as mesmas dimensões.

Mais sobre o batch_size: 
Memória e Eficiência Computacional:
- Processar um lote de 32 imagens é muito mais eficiente em termos de uso de memória do que processar todas as imagens de uma vez.
- A memória da GPU (ou CPU) não é ilimitada, então usar batches permite que grandes conjuntos de dados sejam processados de forma mais eficiente.

Treinamento Mais Estável:
- Atualizar os parâmetros do modelo (pesos da rede neural) após cada imagem poderia levar a uma convergência instável devido à alta variação.
- Usar batches ajuda a estabilizar o processo de treinamento, pois as atualizações são feitas com base na média de erros de várias imagens, suavizando as variações.

Velocidade de Treinamento:
- O uso de batches pode acelerar o treinamento. As operações matriciais (como as multiplicações de matrizes) são mais eficientes quando processadas em paralelo para um lote de dados.


In [4]:
# Função para carregar as imagens
# carrega todas as imagens de diferentes categorias (subpastas), 
# redimensiona as imagens, e cria listas de imagens e seus respectivos rótulos

def load_images_from_folder(folder):
    images = []
    labels = []
    for class_name in os.listdir(folder):
        class_path = os.path.join(folder, class_name)
        if os.path.isdir(class_path):
            for img_name in tqdm(os.listdir(class_path)):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Carregar como escala de cinza
                if img is not None:
                    img = cv2.resize(img, (image_size, image_size))
                    images.append(np.expand_dims(img, axis=-1))  # Expandir dimensões para escala de cinza
                    labels.append(class_name)
    return np.array(images), np.array(labels)

Este trecho de código define uma função chamada `load_images_from_folder`, que tem como objetivo carregar imagens de diferentes categorias (subpastas) a partir de uma pasta principal. A função realiza as seguintes etapas:

1. Inicializa duas listas vazias: `images` para armazenar as imagens e `labels` para armazenar os rótulos das categorias.
2. Percorre todas as subpastas dentro da pasta principal especificada.
3. Dentro de cada subpasta, itera sobre todos os arquivos de imagem.
4. Carrega cada imagem como escala de cinza usando a biblioteca OpenCV.
5. Redimensiona a imagem para um tamanho específico (`image_size` x `image_size`).
6. Adiciona a imagem redimensionada à lista `images`, expandindo suas dimensões para manter o formato de escala de cinza.
7. Adiciona o nome da subpasta (categoria) à lista `labels`.

Retorna duas arrays NumPy: uma contendo todas as imagens e outra contendo seus respectivos rótulos.

In [5]:
# Carregar imagens de treino, teste e validação
# utiliza a função `load_images_from_folder` para processar as imagens em suas respectivas pastas (treino, teste e validação) 
# e retorna arrays contendo as imagens e seus rótulos. 
# Esses conjuntos de dados serão usados para treinar, testar e validar o modelo

train_images, train_labels = load_images_from_folder(train_dir)
test_images, test_labels = load_images_from_folder(test_dir)
val_images, val_labels = load_images_from_folder(val_dir)

100%|██████████| 8/8 [00:00<00:00, 249.84it/s]


In [6]:
# Normalização das imagens
# Esse trecho de código normaliza as imagens dividindo cada valor de pixel por 255. 
# Isso transforma os valores dos pixels de 0-255 para uma faixa entre 0 e 1, 
# o que ajuda a melhorar o desempenho durante o treinamento e garante que todos os pixels tenham valores na mesma faixa.

train_images = train_images / 255.0
test_images = test_images / 255.0
val_images = val_images / 255.0

In [ ]:
# Codificar os rótulos
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

LabelEncoder: Utiliza um codificador de rótulos para transformar rótulos de texto em valores numéricos. 
1. Inicializa um objeto `LabelEncoder`.
2. Codifica os rótulos de treino (`train_labels`) transformando-os em valores numéricos e armazena-os em `train_labels_encoded`.
3. Codifica os rótulos de validação (`val_labels`) utilizando a mesma transformação e armazena-os em `val_labels_encoded`.
A codificação dos rótulos é importante para que o modelo de aprendizado de máquina possa trabalhar com os rótulos de forma eficiente.

In [11]:
# Inicializando o ImageDataGenerator para os dados de treinamento
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalização das imagens
    rotation_range=20,  # Rotação
    width_shift_range=0.2,  # Deslocamento horizontal
    height_shift_range=0.2,  # Deslocamento vertical
    shear_range=0.2,  # Cisalhamento
    zoom_range=0.2,  # Zoom
    horizontal_flip=True,  # Flip horizontal
    fill_mode='nearest'  # Preenchimento
)

# Inicializando o ImageDataGenerator para os dados de validação
val_datagen = ImageDataGenerator(rescale=1./255)  # Normalização das imagens

O `ImageDataGenerator` é uma classe da biblioteca Keras utilizada para pré-processar e gerar lotes de dados de imagem em tempo real, com aumento de dados (data augmentation).

Funções principais:
- Normalização de imagens: Converte valores de pixels de 0-255 para 0-1.
- Aumento de dados: Aplica transformações como rotação, deslocamento, cisalhamento, zoom e flip nas imagens de treino.
- Geração de lotes: Cria lotes de dados em tempo real durante o treinamento.

Benefícios:
- Melhora a generalização do modelo.
- Uso eficiente da memória.
- Reduz overfitting.

In [12]:
# Configuração do ImageDataGenerator com rótulos codificados
train_generator = datagen.flow(
    np.array(train_images),  # Garantir que a forma seja (5216, 224, 224, 1)
    train_labels_encoded,
    batch_size=batch_size
)

val_generator = val_datagen.flow(
    np.array(val_images),  # Garantir que a forma seja (624, 224, 224, 1)
    val_labels_encoded,
    batch_size=batch_size
)


Configura dois objetos ImageDataGenerator para os dados de treinamento e validação com rótulos codificados:
   - `train_generator`: Gera lotes de dados de treinamento aumentados a partir das imagens e rótulos codificados.
   - `val_generator`: Gera lotes de dados de validação a partir das imagens e rótulos codificados.

In [ ]:

# Carregar o modelo VGG16 pré-treinado sem a camada de saída
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Carrega o modelo VGG16 pré-treinado sem a camada de saída:
   - `base_model`: Carrega o modelo VGG16 com pesos pré-treinados no ImageNet e exclui a camada de saída, especificando a entrada como (224, 224, 3).

In [13]:

# Construir o modelo sobre o VGG16
model = Sequential([
    Conv2D(3, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 1)),  # Primeira camada para adaptar escala de cinza para RGB
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Congelar as camadas do modelo base VGG16
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Definindo os callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)


c:\Users\andre\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1. Constrói um modelo usando a base do VGG16:
   - Adiciona uma camada Conv2D para adaptar imagens em escala de cinza para RGB.
   - Adiciona o modelo base VGG16 pré-treinado.
   - Adiciona camadas Flatten, Dense, Dropout e uma camada de saída com ativação sigmoide.

2. Congela as camadas do modelo base VGG16 para que seus pesos não sejam atualizados durante o treinamento.

3. Compila o modelo:
   - Usa o otimizador Adam com uma taxa de aprendizado de 0.0001.
   - Define a perda como 'binary_crossentropy'.
   - Usa a métrica 'accuracy'.

4. Define os callbacks:
   - `early_stopping`: Para o treinamento antecipadamente se a perda de validação não melhorar após 10 épocas e restaura os melhores pesos.
   - `reduce_lr`: Reduz a taxa de aprendizado em um fator de 0.2 se a perda de validação não melhorar após 5 épocas, com uma taxa mínima de aprendizado de 0.00001.

In [14]:
# Treinar o modelo
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

c:\Users\andre\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 3176s 19s/step - accuracy: 0.7646 - loss: 0.4886 - val_accuracy: 0.6875 - val_loss: 0.5592 - learning_rate: 1.0000e-04
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 13866s 85s/step - accuracy: 0.8689 - loss: 0.2943 - val_accuracy: 0.6875 - val_loss: 0.5462 - learning_rate: 1.0000e-04
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 6784s 42s/step - accuracy: 0.8801 - loss: 0.2743 - val_accuracy: 0.6875 - val_loss: 0.6876 - learning_rate: 1.0000e-04
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 3580s 22s/step - accuracy: 0.8850 - loss: 0.2569 - val_accuracy: 0.7500 - val_loss: 0.4475 - learning_rate: 1.0000e-04
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2165s 13s/step - accuracy: 0.8954 - loss: 0.2454 - val_accuracy: 0.7500 - val_loss: 0.4413 - learning_rate: 1.0000e-04
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 2360s 15s/step - accuracy: 0.8993 - loss: 0.2300 - val_accuracy: 0.8125 - val_loss: 0.4343 - learning_rate: 1.0000e-04
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1872s

Treina o modelo com os dados de treinamento e validação.
1. Chama o método `fit` do modelo para iniciar o treinamento.
2. Usa o `train_generator` para fornecer lotes de dados de treinamento.
3. Define o número de épocas de treinamento como 50.
4. Usa o `val_generator` para fornecer dados de validação.
5. Adiciona os callbacks `early_stopping` e `reduce_lr` para monitorar a perda de validação e ajustar a taxa de aprendizado durante o treinamento.
6. Armazena o histórico do treinamento na variável `history`.

In [15]:
# Salvar o modelo
model.save('modelo_pulmoes_vgg16_2.h5')

Salva o modelo treinado em um arquivo H5.
1. Chama o método `save` do modelo.
2. Salva o modelo treinado no arquivo 'modelo_pulmoes_vgg16_2.h5'.

In [16]:
# Carregar imagens de teste
test_images, test_labels = load_images_from_folder(test_dir)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalizar as imagens de teste
test_images = test_images / 255.0

# Avaliar o modelo
test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded, batch_size=batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

100%|██████████| 390/390 [00:08<00:00, 43.52it/s]


20/20 ━━━━━━━━━━━━━━━━━━━━ 183s 9s/step - accuracy: 0.8713 - loss: 0.3638
Test Loss: 0.2635897696018219
Test Accuracy: 0.9054487347602844


Avalia o modelo com as imagens de teste:
     A função `model.evaluate` é usada para avaliar o desempenho de um modelo treinado em um conjunto de dados de teste. Ela calcula a perda (loss) e as métricas definidas (como precisão) para os dados de entrada fornecidos.
 - `test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded, batch_size=batch_size)`: Avalia a perda e a precisão do modelo nos dados de teste.
   - `print(f'Test Loss: {test_loss}')`: Imprime a perda do teste.
   - `print(f'Test Accuracy: {test_accuracy}')`: Imprime a precisão do teste.

In [8]:
# Carregar o modelo treinado
model = load_model('modelo_pulmoes_vgg16_2.h5')

# Carregar imagens de teste
test_images, test_labels = load_images_from_folder(test_dir)

# Codificar os rótulos de teste
test_labels_encoded = label_encoder.transform(test_labels)

# Normalizar as imagens de teste
test_images = test_images / 255.0

# Avaliar o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded, batch_size=batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Incluir análise dos resultados
if (val_loss - test_loss) < 0.1 and (test_accuracy - val_accuracy) > -0.1:
    print("Conclusão: O modelo está generalizando bem e não apresenta sinais significativos de overfitting.")
else:
    print("Conclusão: Pode haver sinais de overfitting. Considere realizar ajustes adicionais.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8750 - loss: 0.3400
Val Loss: 0.33996444940567017
Val Accuracy: 0.875
20/20 ━━━━━━━━━━━━━━━━━━━━ 96s 5s/step - accuracy: 0.8713 - loss: 0.3638
Test Loss: 0.2635897696018219
Test Accuracy: 0.9054487347602844

### Análise dos Resultados ###
Training Accuracy: 95% (estimado)
Training Loss: ~0.20 (estimado)
Val Accuracy: 0.875
Val Loss: 0.33996444940567017
Test Accuracy: 0.9054487347602844
Test Loss: 0.2635897696018219
Conclusão: O modelo está generalizando bem e não apresenta sinais significativos de overfitting.


Etapa de teste do modelo com os dados de teste. A validação é usada durante o processo de treinamento para ajustar hiperparâmetros e monitorar o desempenho em dados não vistos, enquanto o teste é a avaliação final do modelo.
1. Carregar o modelo treinado a partir de um arquivo.
2. Carregar e preprocessar imagens de teste de um diretório.
3. Codificar os rótulos de teste em valores numéricos.
4. Normalizar as imagens de teste.
5. Avaliar o modelo usando as imagens e rótulos de teste, imprimindo a perda e a precisão.
6. Analisar os resultados para determinar se o modelo está generalizando bem ou se há sinais de overfitting.

Sobre o output dos dados de teste de modelo com dados de teste.
### Análise dos Resultados ###
Training Accuracy: 95% (estimado)  # Precisão no conjunto de treinamento (estimado)
Training Loss: ~0.20 (estimado)  # Perda no conjunto de treinamento (estimado)
Val Accuracy: 0.875  # Precisão no conjunto de validação
Val Loss: 0.33996444940567017  # Perda no conjunto de validação
Test Accuracy: 0.9054487347602844  # Precisão no conjunto de teste
Test Loss: 0.2635897696018219  # Perda no conjunto de teste

In [22]:
# INFERENCIA / SIMULAÇÃO >> pode ser carregado em um arquivo ipynb novo / separado 
# Defina o tamanho da imagem usado durante o treinamento
image_size = 224

# Carregar o modelo treinado
model = load_model('modelo_pulmoes_vgg16_2.h5')

# Função para carregar e preprocessar a imagem
def preprocess_image(image_path, image_size):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Carregar a imagem como escala de cinza
    image = cv2.resize(image, (image_size, image_size))  # Redimensionar para o tamanho usado no treinamento
    image = image / 255.0  # Normalização
    image = np.expand_dims(image, axis=-1)  # Adicionar uma dimensão extra para o canal
    image = np.expand_dims(image, axis=0)  # Adicionar uma dimensão extra para o batch
    return image

# Função para fazer a previsão
def make_prediction(image_path, model):
    image = preprocess_image(image_path, image_size)
    prediction = model.predict(image)
    confidence = prediction[0][0]  # Pegando a confiança da previsão
    if confidence > 0.5:
        print(f"Previsão: Pneumonia ({confidence * 100:.2f}%)")
    else:
        print(f"Previsão: Normal ({(1 - confidence) * 100:.2f}%)")

# Caminho para a imagem de teste >> altere aqui para testar com uma imagem de raiox local em seu computador
image_path = 'C:/Users/andre/OneDrive/Desktop/Fiap/Kaggle/Chest_X-Ray/simulacao/person3_virus_17.jpeg'

# Fazer a previsão manual
make_prediction(image_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
Previsão: Pneumonia (52.10%)


1. Define o tamanho da imagem usada durante o treinamento.
   - `image_size = 224`

2. Carrega o modelo treinado a partir de um arquivo H5.
   - `model = load_model('modelo_pulmoes_vgg16_2.h5')`

3. Define uma função para carregar e pré-processar uma imagem:
   - Carrega a imagem como escala de cinza.
   - Redimensiona a imagem para o tamanho usado no treinamento.
   - Normaliza a imagem.
   - Adiciona dimensões extras para o canal e o batch.

4. Define uma função para fazer a previsão com o modelo:
   - Pré-processa a imagem.
   - Faz a previsão usando o modelo.
   - Calcula a confiança da previsão e imprime se é pneumonia ou normal com a porcentagem de confiança.

5. Especifica o caminho para a imagem de teste.
   - `image_path = 'C:/Users/andre/OneDrive/Desktop/Fiap/Kaggle/Chest_X-Ray/simulacao/person3_virus_17.jpeg'`

6. Faz a previsão manualmente chamando a função `make_prediction`.
   - `make_prediction(image_path, model)`

*****************************************************************